In [65]:
import pandas as pd
import sidetable
import regex as re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

Vamos a trabajar con el csv del pair de Limpieza III e intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. 
- Es el momento de eliminar los nulos:


In [66]:
df = pd.read_csv("datos/attacks_pandas_10.csv", index_col= 0)

Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.


In [67]:
imputer = SimpleImputer(strategy="mean", missing_values=np.nan,)

In [68]:
imputer = imputer.fit(df[["age"]])

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '60s'

In [ ]:
df["age"]= imputer.transform(df[["age"]])

NotFittedError: This SimpleImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
df.sample()

,case_number,year,country,area,location,sex_,age,species_,siglo,injury,date,mes,fatal_(y/n)_limpio,sex,country_
227,2013.04.02.a,2013.0,usa,Hawaii,Kaanapali Shores,M,58,4' shark,siglo XXI,NaN,NaN,NaN,NaN,M,usa


Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.
💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.


In [69]:
# Tenemos dos columnas de sex, la que ya habíamos limpiado anteriormente en Pandas sin nulos, y la que no estaba limpia "sex_". Usamos esa.

df.describe(include="object")

,case_number,country,area,location,sex_,age,species_,siglo,injury,date,mes,fatal_(y/n)_limpio,sex,country_
count,7788,1662,1626,1621,1658,1518,1546,1502,6258,6286,5394,5678,7789,1662
unique,6287,95,236,1310,3,94,856,2,3729,5431,22,2,2,95
top,1959.11.22,usa,Florida,"New Smyrna Beach, Volusia County",M,19,White shark,siglo XXI,FATAL,1957,Jul,N,M,usa
freq,2,741,346,73,1435,70,76,785,801,11,620,4292,4519,741


In [70]:
imputer2 = SimpleImputer(strategy="most_frequent", missing_values=np.nan,)

In [71]:
imputer2 = imputer2.fit(df[["sex_"]])

In [72]:
df["sex_"]= imputer2.transform(df[["sex_"]])

In [73]:
df["sex_"].isnull().sum()

0

In [74]:
df.describe(include="object")

,case_number,country,area,location,sex_,age,species_,siglo,injury,date,mes,fatal_(y/n)_limpio,sex,country_
count,7788,1662,1626,1621,7789,1518,1546,1502,6258,6286,5394,5678,7789,1662
unique,6287,95,236,1310,3,94,856,2,3729,5431,22,2,2,95
top,1959.11.22,usa,Florida,"New Smyrna Beach, Volusia County",M,19,White shark,siglo XXI,FATAL,1957,Jul,N,M,usa
freq,2,741,346,73,7566,70,76,785,801,11,620,4292,4519,741


Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.


In [75]:
# Igual que en el pair anterior, como interpretamos que para probar las hipótesis no necesitábamos de la columna type la borramos en limpieza 1, por tanto haremos
# el ejercicio en la columna "injury"

imputer3 = SimpleImputer(strategy="most_frequent", missing_values=np.nan,)
imputer3 = imputer3.fit(df[["injury"]])
df["injury"]= imputer3.transform(df[["injury"]])


In [76]:
df["injury"].isnull().sum()

0

In [77]:
df.describe(include="object")

,case_number,country,area,location,sex_,age,species_,siglo,injury,date,mes,fatal_(y/n)_limpio,sex,country_
count,7788,1662,1626,1621,7789,1518,1546,1502,7789,6286,5394,5678,7789,1662
unique,6287,95,236,1310,3,94,856,2,3729,5431,22,2,2,95
top,1959.11.22,usa,Florida,"New Smyrna Beach, Volusia County",M,19,White shark,siglo XXI,FATAL,1957,Jul,N,M,usa
freq,2,741,346,73,7566,70,76,785,2332,11,620,4292,4519,741


Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.


In [79]:
df_copia1 = df.copy()

In [80]:
df_numericas= df_copia1.select_dtypes(include=np.number)

In [81]:
imputerKNN = KNNImputer(n_neighbors=5)
imputerKNN.fit(df_numericas)

KNNImputer()

In [82]:
numericas_KNN = imputerKNN.transform(df_numericas)

In [83]:
df_KNN_imputer = pd.DataFrame(numericas_KNN, columns= df_numericas.columns)

In [84]:
df_KNN_imputer.head()

,year
0,2018.0
1,2018.0
2,2018.0
3,2018.0
4,2018.0


In [85]:
columnas_df_KNN_imputer = df_KNN_imputer.columns

In [86]:
df_copia1.drop(columnas_df_KNN_imputer,axis=1,inplace=True)

In [87]:
df_copia1[columnas_df_KNN_imputer] = df_KNN_imputer

In [88]:
df_copia1["year"].isnull().sum()

0

Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.


In [89]:
df_copia2 = df.copy()

In [90]:
numericas_iterative = df_copia2.select_dtypes(include= np.number)

In [91]:
i_imputer = IterativeImputer(n_nearest_features= None, imputation_order= "ascending")

In [92]:
i_imputer.fit(numericas_iterative)

IterativeImputer()

In [93]:
i_imputer.transform(numericas_iterative)

array([[2018.        ],
       [2018.        ],
       [2018.        ],
       ...,
       [1832.68839713],
       [1832.68839713],
       [1832.68839713]])

In [94]:
df_numericas_trans = pd.DataFrame(i_imputer.transform(numericas_iterative), columns= numericas_iterative.columns)

In [95]:
df_numericas_trans.head()

,year
0,2018.0
1,2018.0
2,2018.0
3,2018.0
4,2018.0


In [96]:
df_numericas_trans.isnull().sum()

year    0
dtype: int64

In [97]:
nombres = df_numericas_trans.columns

In [98]:
df_copia2.drop(nombres,axis=1,inplace=True)

In [99]:
df_copia2[nombres] = df_numericas_trans

In [100]:
df_copia2.isnull().sum()

case_number              1
country               6127
area                  6163
location              6168
sex_                     0
age                   6271
species_              6243
siglo                 6287
injury                   0
date                  1503
mes                   2395
fatal_(y/n)_limpio    2111
sex                      0
country_              6127
year                     0
dtype: int64

¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?


Por lo que podemos observar el Iterative Imputer se basa en los datos cercanos para crear una predicción que se ajuste a lo que observa de los datos preexistentes, y el KNN busca como patrones en los datos para agruparlos y meter el nan en el grupo al que más se asemeje.

Aparte el Iterative Imputer podemos pasarle más parámetros para controlar el resultado (que sea la moda, la medias, mediana...) y el orden de sustitución.

Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

In [101]:
# Como consideramos que el KNN y el II  han hehco un trabajo similar guardamos solo el último de los df donde hemos reemplazado nulos

df_copia2.to_csv("datos/attacks_pandas_sklearn.csv")

In [102]:
holi = "😊"